In [1]:
from graphics import * 
import pandas as pd 
import numpy as np 
from new_data_loader import bench, squat, deadlift
from data_loader import be, sq, de
import plotly.graph_objects as go
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 

interval = 100
k = 5
t = squat[0]['gravityX'].dropna().to_numpy()
bench.extend(be)
squat.extend(sq)
deadlift.extend(de)

bench = [e for e in bench if len(e['gravityX'].dropna().to_numpy()) > interval]
squat = [e for e in squat if len(e['gravityX'].dropna().to_numpy()) > interval]
deadlift = [e for e in deadlift if len(e['gravityX'].dropna().to_numpy()) > interval]

['novel', 'bench', '20', '5', 'bad-2024-10-25', '23-24-47.json']
['brian', 'bench', '35', '5', '-2024-10-25', '23-19-10.json']
['novel', 'squat', '20', '5', '-2024-10-25', '00-49-17.json']
['neal', 'bench', '0', '5', '-2024-10-25', '23-34-22.json']
['novel', 'squat', '20', '5', '-2024-10-25', '00-50-36.json']
['neal', 'deadlift', '25', '5', '-2024-10-25', '23-50-52.json']
['novel', 'deadlift', '35', '5', '-2024-10-25', '00-11-12.json']
['brian', 'deadlift', '60', '5', '-2024-10-25', '23-48-04.json']
['novel', 'deadlift', '35', '5', '-2024-10-25', '00-12-44.json']
['brian', 'bench', '45', '5', '-2024-10-25', '23-39-37.json']
['brian', 'squat', '35', '5', '-2024-10-25', '00-22-02.json']
['novel', 'deadlift', '25', '5', '-2024-10-25', '00-09-36.json']
['novel', 'deadlift', '40', '5', '-2024-10-25', '00-14-48.json']
['brian', 'deadlift', '35', '5', '-2024-10-25', '23-43-11.json']
['neal', 'bench', '10', '5', '-2024-10-25', '23-34-56.json']
['neal', 'squat', '15', '4', '-2024-10-25', '00-39

In [2]:
def process_signal(sig, interval, normalize=False):
    if normalize: 
        mean = np.mean(sig)
        std = np.std(sig)
        sig = (sig - mean)/std
        pos = np.where(sig > 0, 1, 0)
        zeros = pos[1:] - pos[:-1]
        zeros = np.where(sig != 0)
        zeros = np.average(np.diff(zeros))
        interval = zeros
    if interval not in sig.shape:
        sig = sig.reshape(-1, 1)
    pad_len = interval - (sig.shape[0] % interval)
    if pad_len != interval:
        padding = np.zeros((pad_len, sig.shape[1]))
        padding[:, :] = sig[-1, -1]
        sig = np.concatenate([sig, padding], axis=0)
    sig = sig.reshape(-1, interval, sig.shape[1])
    sig = np.squeeze(sig)
    return sig

def get_mean_std_tensor(sig, interval, normalize=False):
    temp = process_signal(sig, interval, normalize)
    tensor = np.vstack([np.average(temp, axis=0), np.std(temp, axis=0)])
    tensor = np.average(tensor, axis=1)
    return tensor

In [3]:
train_size = 5
labels = ['gravityX', 'gravityY', 'gravityZ']

In [4]:
train_bench = bench[:train_size]
train_squat = squat[:train_size]
train_deadlift = deadlift[:train_size]

train_X_bench = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in train_bench]
train_X_squat = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in train_squat]
train_X_deadlift = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in train_deadlift]

train_X = np.array(train_X_bench + train_X_squat + train_X_deadlift)


train_y = ['bench' for e in train_bench]
train_y += ['squat' for e in train_squat]
train_y += ['deadlift' for e in train_deadlift]
train_y = np.array(train_y)

In [5]:
np.array(train_X).shape

(15, 2, 3)

In [20]:
test_bench = bench[train_size:]
test_squat = squat[train_size:]
test_deadlift = deadlift[train_size:]

test_X_bench = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in test_bench]
test_X_squat = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in test_squat]
test_X_deadlift = [np.array([get_mean_std_tensor(e[l].dropna().to_numpy(), interval) for l in labels]).T for e in test_deadlift]

test_X = np.array(test_X_bench + test_X_squat + test_X_deadlift)

test_y = ['bench' for e in test_bench]
test_y += ['squat' for e in test_squat]
test_y += ['deadlift' for e in test_deadlift]
test_y = np.array(test_y)

In [21]:
dist = [np.sum((e - test_X[0])**2) for e in train_X]
min_idx = np.argpartition(dist, k)[:k]
unique, counts = np.unique(train_y[min_idx], return_counts=True)
res = unique[counts == counts.max()]
print("Most frequent numbers are : " + str(res) + " with count " + str(counts.max()) + ' Actual: ' + train_y[0])

Most frequent numbers are : ['bench'] with count 4 Actual: bench


In [22]:
total = 0
acc = 0
for t, label in zip(test_X, test_y):
    dist = [np.sum((e - t)**2) for e in train_X]
    min_idx = np.argpartition(dist, k)[:k]
    unique, counts = np.unique(train_y[min_idx], return_counts=True)
    res = unique[counts == counts.max()]
    print("Most frequent numbers are : " + str(res) + " with count " + str(counts.max()) + ' Actual: ' + label)
    if res[0] == label: 
        acc += 1

    total += 1

acc/total

Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are

1.0

In [9]:

total = 0
acc = 0
for t, label in zip(test_X, test_y):
    dist = [np.sum((e - t)**2) for e in train_X]
    min_idx = np.argpartition(dist, k)[:k]
    unique, counts = np.unique(train_y[min_idx], return_counts=True)
    res = unique[counts == counts.max()]
    print("Most frequent numbers are : " + str(res) + " with count " + str(counts.max()) + ' Actual: ' + label)
    if res[0] == label: 
        acc += 1

    total += 1

acc/total

Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 4 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are : ['bench'] with count 5 Actual: bench
Most frequent numbers are

1.0

In [10]:
def eye_diag(sig, interval):

    fig = go.Figure()
    print(sig.shape)
    for e in sig:
        fig.add_trace(
            go.Scatter(x=list(range(len(e))), y=e)
        )
    
    fig.update_layout(
        height=600, 
        title="Gravity Signals Over Time",
        xaxis_title="Time Steps",
        hovermode="x unified", 
    )
    
    fig.show()



In [11]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter

# Parameters
segment_len = 100
axes = ['gravityX', 'gravityY', 'gravityZ']

def get_segments(df, label):
    segments = []
    labels = []
    min_len = min([len(df[axis].dropna()) for axis in axes])
    max_chunks = min_len // segment_len

    for i in range(max_chunks):
        chunk = []
        for axis in axes:
            vals = df[axis].dropna().values[i*segment_len : (i+1)*segment_len]
            chunk.append(vals)
        segment = np.concatenate(chunk)
        segments.append(segment)
        labels.append(label)
    return segments, labels

# Gather all segments
X, y = [], []
for df in train_bench:
    segs, labels = get_segments(df, 'bench')
    X.extend(segs)
    y.extend(labels)

for df in train_squat:
    segs, labels = get_segments(df, 'squat')
    X.extend(segs)
    y.extend(labels)

for df in train_deadlift:
    segs, labels = get_segments(df, 'deadlift')
    X.extend(segs)
    y.extend(labels)

X = np.array(X)
y = np.array(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=42)

# Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       bench       1.00      0.41      0.58        68
    deadlift       0.73      0.99      0.84       103
       squat       0.95      1.00      0.97        77

    accuracy                           0.83       248
   macro avg       0.89      0.80      0.80       248
weighted avg       0.87      0.83      0.81       248



In [12]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter

# Parameters
segment_len = 100
axes = ['gravityX', 'gravityY', 'gravityZ']

def get_segments(df, label):
    segments = []
    labels = []
    min_len = min([len(df[axis].dropna()) for axis in axes])
    max_chunks = min_len // segment_len

    for i in range(max_chunks):
        chunk = []
        for axis in axes:
            vals = df[axis].dropna().values[i*segment_len : (i+1)*segment_len]
            mean = np.mean(vals)
            std = np.std(vals)
            vals = (vals - mean)/std

            chunk.append(vals)
        segment = np.concatenate(chunk)
        segments.append(segment)
        labels.append(label)
    return segments, labels

# Gather all segments
X, y = [], []
for df in train_bench:
    segs, labels = get_segments(df, 'bench')
    X.extend(segs)
    y.extend(labels)

for df in train_squat:
    segs, labels = get_segments(df, 'squat')
    X.extend(segs)
    y.extend(labels)

for df in train_deadlift:
    segs, labels = get_segments(df, 'deadlift')
    X.extend(segs)
    y.extend(labels)

X = np.array(X)
y = np.array(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=42)

# Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       bench       0.00      0.00      0.00        68
    deadlift       0.45      0.26      0.33       103
       squat       0.30      0.73      0.42        77

    accuracy                           0.33       248
   macro avg       0.25      0.33      0.25       248
weighted avg       0.28      0.33      0.27       248



/Users/neal/Documents/python_projects/senior_design/.env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/neal/Documents/python_projects/senior_design/.env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/neal/Documents/python_projects/senior_design/.env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control th

- dropout layer in CNN 
- average x intercept distance for half period in eye diagram 

